# Automatisation d'emails

Objectif : envoyer un email avec du contenu provenant d'internet

https://www.youtube.com/watch?v=s8XjEuplx_U

Librairies dont on va avoir besoin :
- requests
- bs4
- smtplib (default)
- email.mime (default)
- datetime (default)

In [91]:
import requests

In [92]:
import bs4
import smtplib
import email.mime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import datetime

In [94]:
from bs4 import BeautifulSoup

In [95]:
now = datetime.datetime.now()

In [96]:
now

datetime.datetime(2021, 11, 5, 14, 56, 26, 694682)

In [97]:
# email content placeholder
content = ' '

In [98]:
# extracting Hacker News Stories

def extract_news(url):
    print('Extracting Hacker News Stories...')
    cnt = ''
    cnt +=('<b>HN Top Stories:</b>\n'+'<br>'+'-'*50+'<br>')
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content,'html.parser')
    for i,tag in enumerate(soup.find_all('td',attrs={'class':'title','valign':''})):
        cnt += ((str(i+1)+' :: '+tag.text + "\n" + '<br>') if tag.text!='More' else '')
        #print(tag.prettify) #find_all('span',attrs={'class':'sitestr'}))
    return(cnt)

In [99]:
cnt = extract_news('https://news.ycombinator.com/')

Extracting Hacker News Stories...


In [100]:
content

' '

In [101]:
content += cnt

In [102]:
content += ('<br>------<br>')
content +=('<br><br>End of Message')

In [103]:
content

" <b>HN Top Stories:</b>\n<br>--------------------------------------------------<br>1 :: Pfizer's oral Covid-19 antiviral cuts hospitalization, death by 85% (fiercebiotech.com)\n<br>2 :: Launch HN: Terra (YC W21) – API to easily connect wearables to apps (tryterra.co)\n<br>3 :: Why is Excalidraw so good? (offbyone.us)\n<br>4 :: Peeking Through Logs (madhouse-project.org)\n<br>5 :: So you want to Scrape like the Big Boys? (incolumitas.com)\n<br>6 :: The “Ultimate” ThinkPad (2020) (calbryant.uk)\n<br>7 :: React Aria: A headless UI component library (adobe.com)\n<br>8 :: From macOS to Arch Linux (juxt.pro)\n<br>9 :: Leipzig: A walk around a city reborn (literaryreview.co.uk)\n<br>10 :: Factored Quality (YC W20) Is Hiring a Front End Engineer (4 Years Experience)\n<br>11 :: Show HN: View the patent and innovation history of any company (goodip.io)\n<br>12 :: Wasp: WASM Programming Language (github.com/pannous)\n<br>13 :: How to learn compilers: LLVM Edition (lowlevelbits.org)\n<br>14 :: Ca

### A présent, il faut envoyer le mail

In [104]:
# update email details

In [116]:
SERVER = 'smtp.gmail.com' # your smtp server
PORT = 587 # port number
FROM = '********' # email adress
TO = '*******' # destination email adress
PASS = '*****' # password

Il faut créer l'email en MIMEMultipart

In [106]:
msg = MIMEMultipart()

In [107]:
msg['Subject'] =  'Top News Stories HN' + ' ' + str(now.day) + '-' + str(now.month) + '-' + str(
    now.year)

In [108]:
msg['From'] = FROM

In [109]:
msg['To'] = TO

In [110]:
msg.attach(MIMEText(content, 'html'))

On s'occupe du server à présent

In [111]:
server = smtplib.SMTP(SERVER, PORT)

Avant d'envoyer le mail il faut se rendre dans le google account pour modifier quelques paramètres
https://myaccount.google.com/security
Accès moins sécurisé des applications -> Turn On

In [112]:
#server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server.set_debuglevel(1)
server.ehlo()
server.starttls()
#server.ehlo

send: 'ehlo pp182.ad.phedre.fr\r\n'
reply: b'250-smtp.gmail.com at your service, [37.1.241.158]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [37.1.241.158]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'


(220, b'2.0.0 Ready to start TLS')

In [113]:
server.login(FROM, PASS)

send: 'ehlo pp182.ad.phedre.fr\r\n'
reply: b'250-smtp.gmail.com at your service, [37.1.241.158]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [37.1.241.158]\nSIZE 35882577\n8BITMIME\nAUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'AUTH PLAIN AHBpZXJyZWxvdXBnYXJyaWd1ZXNAZ21haWwuY29tAENAbnlvbjkx\r\n'
reply: b'235 2.7.0 Accepted\r\n'
reply: retcode (235); Msg: b'2.7.0 Accepted'


(235, b'2.7.0 Accepted')

In [114]:
server.sendmail(FROM, TO, msg.as_string())

send: 'mail FROM:<pierreloupgarrigues@gmail.com> size=3387\r\n'
reply: b'250 2.1.0 OK j10sm7995219wru.15 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.0 OK j10sm7995219wru.15 - gsmtp'
send: 'rcpt TO:<pierre-loupg@hotmail.fr>\r\n'
reply: b'250 2.1.5 OK j10sm7995219wru.15 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.5 OK j10sm7995219wru.15 - gsmtp'
send: 'data\r\n'
reply: b'354  Go ahead j10sm7995219wru.15 - gsmtp\r\n'
reply: retcode (354); Msg: b'Go ahead j10sm7995219wru.15 - gsmtp'
data: (354, b'Go ahead j10sm7995219wru.15 - gsmtp')
send: b'Content-Type: multipart/mixed; boundary="===============5857226457224904349=="\r\nMIME-Version: 1.0\r\nSubject: Top News Stories HN 5-11-2021\r\nFrom: pierreloupgarrigues@gmail.com\r\nTo: pierre-loupg@hotmail.fr\r\n\r\n--===============5857226457224904349==\r\nContent-Type: text/html; charset="utf-8"\r\nMIME-Version: 1.0\r\nContent-Transfer-Encoding: base64\r\n\r\nIDxiPkhOIFRvcCBTdG9yaWVzOjwvYj4KPGJyPi0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0t\r\nLS0tL

{}

In [115]:
server.quit()

send: 'quit\r\n'
reply: b'221 2.0.0 closing connection j10sm7995219wru.15 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection j10sm7995219wru.15 - gsmtp'


(221, b'2.0.0 closing connection j10sm7995219wru.15 - gsmtp')